# 翻译器-BatchNode

In [14]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import time

In [6]:
load_dotenv()

True

In [8]:
def call_llm(prompt):
    client = OpenAI(
        base_url= os.getenv("BASE_URL"),
        api_key=os.getenv("DASHSCOPE_API_KEY")
    )
    res = client.chat.completions.create(
        model = os.getenv("MODEL_NAME"),
        messages = [{
            "role": "user",
            "content": prompt
        }]
    )
    return res.choices[0].message.content

In [17]:
from pocketflow import Node,Flow, BatchNode

class TranslateTextNode(BatchNode):
    def prep(self, shared):
        text = shared.get("text", "(No text provided)")
        languages = shared.get("languages", ["Chinese", "Spanish", "Japanese", "German", 
                              "Russian", "Portuguese", "French", "Korean"])
        
        # Create batches for each language translation
        return [(text, lang) for lang in languages]

    # 也就是说，prep返回多组prep_res,对应每次执行exec会运行一组prep_res 会执行多组
    # 整个过程自动运行
    def exec(self, data_tuple):
        text, language = data_tuple
        
        prompt = f"""
        Please translate the following markdown file into {language}. 
        But keep the original markdown format, links and code blocks.
        Directly return the translated text, without any other text or comments.

        Original: 
        {text}

        Translated:
        """
        
        result = call_llm(prompt)
        print(f"Translated {language} text")
        return {"language": language, "translation": result}

    def post(self, shared, prep_res, exec_res_list):
        # Create output directory if it doesn't exist
        output_dir = shared.get("output_dir", "translations")
        os.makedirs(output_dir, exist_ok=True)
        
        # Write each translation to a file
        for result in exec_res_list:
            language, translation = result["language"], result["translation"]
            
            # Write to file
            filename = os.path.join(output_dir, f"README_{language.upper()}.md")
            with open(filename, "w", encoding="utf-8") as f:
                f.write(translation)
            
            print(f"Saved translation to {filename}")


In [ ]:
translation_node = TranslateTextNode(max_retries=3)
translation_flow = Flow(start=translation_node)



In [ ]:
    with open("./README.md", "r") as f:
        text = f.read()

    shared = {
        "text": text,
        "languages": ["Chinese", "Japanese", "German", "Russian", "French", "Korean"],
        "output_dir": "./data"
    }

    print(f"Starting sequential translation into {len(shared['languages'])} languages...")
    start_time = time.perf_counter()

    translation_flow.run(shared)

    end_time = time.perf_counter()
    duration = end_time - start_time

    print(f"\nTotal sequential translation time: {duration:.4f} seconds") # Print duration
    print("\n=== Translation Complete ===")
    print(f"Translations saved to: {shared['output_dir']}")
    print("============================")

Starting sequential translation into 8 languages...
Translated Chinese text
Translated Spanish text
Translated Japanese text
Translated German text
Translated Russian text
Translated Portuguese text
Translated French text
Translated Korean text
Saved translation to ./data/README_CHINESE.md
Saved translation to ./data/README_SPANISH.md
Saved translation to ./data/README_JAPANESE.md
Saved translation to ./data/README_GERMAN.md
Saved translation to ./data/README_RUSSIAN.md
Saved translation to ./data/README_PORTUGUESE.md
Saved translation to ./data/README_FRENCH.md
Saved translation to ./data/README_KOREAN.md

Total sequential translation time: 61.9263 seconds

=== Translation Complete ===
Translations saved to: ./data
